In [ ]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

from detectron2.data import MetadataCatalog, DatasetCatalog
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

In [ ]:
from shapely.geometry import Polygon
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

if "disease_train_defect_only" in DatasetCatalog.list():
    DatasetCatalog.remove("disease_train_defect_only")
    DatasetCatalog.remove("disease_all_test")

def get_leaf_dicts(directory, min_annotation_area=10000):  # Set a suitable threshold for min_annotation_area
    class_name = 'disease'
    dataset_dicts = []
    
    for idx, filename in enumerate([file for file in os.listdir(directory) if file.endswith('.json')]):
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        name = filename.split('.')
        record["file_name"] = os.path.join(directory, name[0] + '.JPG')
        record["image_id"] = idx
        record["height"] = 4000
        record["width"] = 6000

        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            label = anno['label']
            if label != class_name:
                continue  # Skip annotations of other classes

            px = [a[0] for a in anno['points']]
            py = [a[1] for a in anno['points']]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            # Calculate the annotation area using Shapely
            annotation_polygon = Polygon([(px[i], py[i]) for i in range(len(px))])
            annotation_area = annotation_polygon.area

            # Filter out annotations with small area
            if annotation_area >= min_annotation_area:
                obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": [poly],
                    "category_id": 0,
                    "iscrowd": 0
                }
                objs.append(obj)
        
        if objs:
            record["annotations"] = objs
            dataset_dicts.append(record)
    
    return dataset_dicts


In [ ]:
for d in ["all_train", "all_test"]:
    DatasetCatalog.register("diseases_" + d, lambda d=d: get_leaf_dicts(r'path/to/all_train_and_all_test_folder' + d))
    MetadataCatalog.get("diseases_" + d).set(thing_classes=['disease'])
leaf_metadata = MetadataCatalog.get("diseases_all_train")

In [ ]:
dataset_dicts = get_leaf_dicts(r'/home/pallavi/more_data/data/all_train')

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()

cfg_file_path =r"/path/to/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(cfg_file_path)

In [ ]:
#print(get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("diseases_all_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
print(cfg.MODEL.WEIGHTS)
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 20000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

save_period = 100  # Save every 50 epochs
save_callback = SaveModelCallback(cfg.OUTPUT_DIR, save_period)
trainer = DefaultTrainer(cfg) 
trainer.register_hooks([save_callback])
trainer.resume_or_load(resume=False)
trainer.train()